<h1>PREprocessing от Музыкальных Детективов<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Библиотеки-и-настройки" data-toc-modified-id="Библиотеки-и-настройки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Библиотеки и настройки</a></span><ul class="toc-item"><li><span><a href="#Библиотеки" data-toc-modified-id="Библиотеки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Библиотеки</a></span></li><li><span><a href="#Настройки" data-toc-modified-id="Настройки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Настройки</a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Метрика" data-toc-modified-id="Метрика-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Метрика</a></span></li><li><span><a href="#Модели" data-toc-modified-id="Модели-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Введение

**Это вторая из трех тетрадей: EDA, PREprocessing, MODeling.**

**Задача:**

В этом хакатоне вам предлагается разработать решение, которое:

- **Первая задача:** может классифицировать треки по признаку кавер-некавер;
- **Вторая задача:** связывать (группировать) каверы и исходный трек;
- **Третья задача:** находит исходный трек в цепочке каверов.

**Данные:**

Файл covers.json содержит **разметку каверов**, сделанную редакторами сервиса:

- track_id - уникальный идентификатор трека;
- track_remake_type - метка, присвоенная редакторами. Может принимать значения ORIGINAL и COVER;
- original_track_id - уникальный идентификатор исходного трека.

💡 Обратите внимание, что не для всех каверов известны идентификаторы исходных треков!!!

**Метаинформация**

- track_id - уникальный идентификатор трека;
- dttm - первая дата появления информации о треке;
- title - название трека;
- language - язык исполнения;
- isrc - международный уникальный идентификатор трека;
- genres - жанры;
- duration - длительность трека;

**Текст песен**

- track_id - уникальный идентификатор трека;
- lyricId - уникальный идентификатор текста;
- text - текст трека.

## Библиотеки и настройки

### Библиотеки

In [ ]:
# !pip install ydata_profiling -U

In [ ]:
# ! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_sm

In [ ]:
# ! pip install -q optuna

In [ ]:
# ! pip install lightgbm

In [ ]:
import pandas as pd 

# для анализа данных
from ydata_profiling import ProfileReport

import numpy as np

# для графиков
import matplotlib.pyplot as plt
import seaborn as sns

#для загрузки данных и с сервера и локально
import os
# для скрытия ошибок
import warnings
from IPython.display import display
# для условно рандомных состояний
from numpy.random import RandomState
# регулярные выражения
import re
#Лемматизатор
import spacy
# модель для лемматизации английского языка
import en_core_web_sm

# разбиение на выборки
from sklearn.model_selection import train_test_split
# перемешивание для upsample
from sklearn.utils import shuffle

from datetime import datetime

# # расчитаем TF-IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# from nltk.corpus import stopwords
# import nltk

# Логистическая регрессия
from sklearn.linear_model import LogisticRegression
# # Дерево Решений классификатор
# from sklearn.tree import DecisionTreeClassifier
# # Случайный лес классификатор
# from sklearn.ensemble import RandomForestClassifier
# Ridge регрессия
from sklearn.linear_model import RidgeClassifier
# для создания конвеера/ трудопровода
from sklearn.pipeline import Pipeline, make_pipeline
# для маштабирования признаков
from sklearn.preprocessing import StandardScaler
# заполнение пропусков
from sklearn.impute import SimpleImputer
# векторизация текста
from sklearn.feature_extraction.text import TfidfVectorizer
# обработка категориальных признаков
from sklearn.preprocessing import OrdinalEncoder
# обработка категориальных признаков
from sklearn.preprocessing import OneHotEncoder
# для раздельной предобработки количествнных и категорийных признаков
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier

# F1 метрика
from sklearn.metrics import f1_score

# # для подбора параметров моделей
# from sklearn.model_selection import GridSearchCV

# оптимизация параметров моделей
import optuna

import lightgbm as lgb

### Настройки

In [ ]:
# для скрытия ошибок
warnings.filterwarnings("ignore")

# для увеличения окна вывода 
pd.options.display.max_rows = 100
# максимальная вместимость ячейки для отображения
pd.set_option('display.max_colwidth', None)
# четыре знака после запятой
pd.set_option('display.float_format', '{:.4f}'.format)

# для задания размера графиков по умолчанию
plt.rcParams["figure.figsize"] = (7, 5)

# для псевдорандомных значений
np.random.seed(seed=54321)
np.random.RandomState(seed=54321)
RS=54321

# # загрузим английские стоп слова
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))

## Загрузка данных

In [ ]:
path_features_train = 'data/preprocessing/features_train.csv'

features_train = pd.read_csv(path_features_train)
features_train['dttm'] = pd.to_datetime(features_train['dttm'], unit='ns', origin='unix').astype('int')
features_train.info()
features_train.head()



In [ ]:
path_features_train_upsamled = 'data/preprocessing/features_train_upsamled.csv'

features_train_upsamled = pd.read_csv(path_features_train_upsamled)
features_train_upsamled['dttm'] = pd.to_datetime(
    features_train_upsamled['dttm'], unit='ns', origin='unix').astype('int')
features_train_upsamled.info()
features_train_upsamled.head()

In [ ]:
path_features_valid = 'data/preprocessing/features_valid.csv'

features_valid = pd.read_csv(path_features_valid)
features_valid['dttm'] = pd.to_datetime(features_valid['dttm'], 
                                        unit='ns', origin='unix').astype('int')
features_valid.info()
features_valid.head()

In [ ]:
path_features_test = 'data/preprocessing/features_test.csv'

features_test = pd.read_csv(path_features_test)
features_test['dttm'] = pd.to_datetime(features_test['dttm'], 
                                       unit='ns', origin='unix').astype('int')
features_test.info()
features_test.head()

In [ ]:
path_target_train = 'data/preprocessing/target_train.csv'

target_train = pd.read_csv(path_target_train)
display(target_train.describe())
target_train.head()

In [ ]:
path_target_train_upsampled = 'data/preprocessing/target_train_upsampled.csv'

target_train_upsampled = pd.read_csv(path_target_train_upsampled)
display(target_train_upsampled.describe())
target_train.head()

In [ ]:
path_target_valid = 'data/preprocessing/target_valid.csv'

target_valid = pd.read_csv(path_target_valid)
display(target_valid.describe())
target_valid.head()

In [ ]:
path_target_test = 'data/preprocessing/target_test.csv'

target_test = pd.read_csv(path_target_test)
display(target_test.describe())
target_test.head()

## Метрика

## Модели

In [ ]:
features_train_upsamled.columns

In [ ]:
# запишем обучающие признаки в соответствующие списки
features_list_text = ['title_lemm']
features_list_cat = ['language', 'title_language', 'isrc_country',
                     'isrc_prod_centr', 'genres_big_cat',
#                      'title_lemm'
                    ]

features_list_num = ['dttm', 'duration', 'title_len', 'isrc_year','isrc_reg_number']

In [ ]:
category_features_index = []

for col in features_list_cat:
    category_features_index.append(features_train_upsamled.columns.get_loc(col))
    
category_features_index

In [ ]:
# задаем шаги в Pipeline

# обработка численных признаков для базового алгоритма Линейная регрессия
# заполнение 0 и стандартизация с маштабированием
num_steps_linear = [('imputer',  SimpleImputer(missing_values=np.nan
                                        , strategy='constant'
                                        , fill_value=0
                                        , add_indicator=True
                                        )
                    )
                    , ('scaler', StandardScaler()
                       )
                     ]
num_preprocessor_linear = Pipeline(num_steps_linear)
                     
# обработка численных признаков для базового алгоритма Дерево решений
# заполнение аномальными значениями
num_steps_tree = [('imputer',  SimpleImputer(missing_values=np.nan
                                            , strategy='constant'
                                            , fill_value=-1000
                                            )
                  )
                 ]
num_preprocessor_tree = Pipeline(num_steps_tree)                     

# обработка категориальных признаков для базового алгоритма Линейная регрессия
category_steps_linear = [('imputer', SimpleImputer(missing_values=np.nan, 
                                                   strategy='constant', 
                                                   fill_value='unknown'))
                         , ('encoder', OrdinalEncoder(handle_unknown = 'ignore'))
                         , ('scaler', StandardScaler())
                        ]


category_preprocessor_linear = Pipeline(category_steps_linear)

# обработка категориальных признаков для базового алгоритма Дерово решений
category_steps_tree = [('imputer', SimpleImputer(missing_values=np.nan, 
                                                 strategy='constant', 
                                                 fill_value='unknown'))
                       ,('encoder', OrdinalEncoder(handle_unknown = 'ignore', 
#                                                    dtype=np.uint32
                                                  )
                        )
                      ]
category_preprocessor_tree = Pipeline(category_steps_tree)

# обработка текстовых признаков для линейных алгоритмов
text_step_linear = [ ('vect', TfidfVectorizer()),
#                    ('scaler', StandardScaler()) ?
                   ]
text_preprocessor_linear = Pipeline(text_step_linear)

# обработка текстовых признаков для базового алгоритма Дерово решений
text_step_tree = [ ('vect', TfidfVectorizer())]

text_preprocessor_tree = Pipeline(text_step_tree)


# собираем все вместе для базового алгоритма Линейная регрессия
preprocessor_linear = ColumnTransformer(transformers=[('num', num_preprocessor_linear, features_list_num)
                                                      , ('cat', category_preprocessor_linear, features_list_cat)
#                                                       , ('text', text_preprocessor_linear, features_list_text)
                                                     ]
                                        , remainder='passthrough'
                                       )

# собираем все вместе для базового алгоритма Дерево Решений
preprocessor_tree = ColumnTransformer(transformers=[('num', num_preprocessor_tree, features_list_num),
                                                    ('cat', category_preprocessor_tree, features_list_cat),
#                                                     ('text', text_preprocessor_tree, features_list_text)
                                                   ]
                                      , remainder='passthrough'
                                     )


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000, step=10)
    max_depth = trial.suggest_int('max_depth', 4, 50)
    num_leaves = trial.suggest_int('num_leaves', 10, 150, step=20)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, step=None, log=True)
    
    pipeline_lgbm_classifier = Pipeline([
        ('preprocessor', preprocessor_tree),
        ('classifier', lgb.LGBMClassifier(boosting_type='gbdt',
                                          num_leaves=num_leaves,
                                          max_depth=max_depth,
                                          learning_rate=learning_rate,
                                          objective= 'binary', 
                                          n_estimators=n_estimators,
                                          random_state=RS,
                                          n_jobs=-1,
                                          
#                                           class_weight='balanced'
                                          
                                         )
         )
        ]
        )
#     pipeline_lgbm_classifier.fit(features_train
# #                                  .drop(columns='title_lemm')
#                                  ,
#                                  target_train,
#                                  classifier__categorical_feature=category_features_index
# #                                  classifier__categorical_feature=features_list_cat
#                                  )

#     f_one = f1_score(target_train,
#                      pipeline_lgbm_classifier.predict(features_train
# #                                                       .drop(columns='title_lemm')
#                                                      )
#                     )
    
    pipeline_lgbm_classifier.fit(features_train_upsamled
#                                  .drop(columns='title_lemm')
                                 ,
                                 target_train_upsampled,
                                 classifier__categorical_feature=category_features_index
#                                  classifier__categorical_feature=features_list_cat
                                 )

    f_one = f1_score(target_train_upsampled,
                     pipeline_lgbm_classifier.predict(features_train_upsamled
#                                                       .drop(columns='title_lemm')
                                                     )
                    )
#     f_one = f1_score(target_valid,
#                      pipeline_lgbm_classifier.predict(features_valid
# #                                                       .drop(columns='title_lemm')
#                                                      )
#                     )
    return f_one


study = optuna.create_study(direction='maximize'
                            , sampler=optuna.samplers.RandomSampler(seed=RS)
                            , pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
                           )
study.optimize(objective, n_trials=100, timeout=600)
study.best_trial

In [ ]:
# запишем наш pipeline_lgbm_classifier_best с лучшими параметрами
pipeline_lgbm_classifier_best = Pipeline([
        ('preprocessor', preprocessor_tree),
        ('classifier', lgb.LGBMClassifier(boosting_type='gbdt',
                                          num_leaves=study.best_params['num_leaves'],
                                          max_depth=study.best_params['max_depth'],
                                          learning_rate=study.best_params['learning_rate'],
                                          objective= 'binary', 
                                          n_estimators=study.best_params['n_estimators'],
                                          random_state=RS,
                                          n_jobs=-1,)
         )
        ]
        )
pipeline_lgbm_classifier

In [ ]:
# обучим на сбалансированной выборке
pipeline_lgbm_classifier_best.fit(features_train_upsamled, target_train_upsampled)

In [ ]:
pipeline_lgbm_classifier_best.steps[1]

In [ ]:
# проверим на проверочно выборке
f_one = f1_score(target_valid,
                     pipeline_lgbm_classifier_best.predict(features_valid
#                                                       .drop(columns='title_lemm')
                                                     )
                )
f_one                  

In [ ]:
# проверим на тестовой выборке выборке
f_one = f1_score(target_test,
                 pipeline_lgbm_classifier_best.predict(features_test
#                                                       .drop(columns='title_lemm')
                                                     )
                )
f_one                  

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
dummy_model = DummyClassifier(strategy="most_frequent")
dummy_model.fit(features_train_upsamled, target_train_upsampled)

f_one = f1_score(target_valid, dummy_model.predict(features_valid)
f_one


## Вывод